In [56]:
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import pyarrow as pa
from urllib.parse import quote_plus
from pandas.api.types import is_string_dtype
import gc
import warnings

import config
import config_media_costs
from normalize_funcs import normalize_columns_types, append_custom_columns
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table, removeRowsFromDB
from create_dicts import get_cleaning_dict, get_media_discounts
# from create_dicts_adex import get_adex_dicts

db_name = config.db_name

In [57]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

In [58]:
# создаем функцию, которая вернет номер месяца от начальной даты отсчета
def get_mon_num_from_date(curr_date):
    curr_date = datetime.strptime(str(curr_date), '%Y-%m-%d')
    # с помощью метода relativedelta получаем кол-во лет и месяцев от даты начала до текущей даты
    relative_date = relativedelta(curr_date, config_media_costs.start_of_the_time)
    # теперь нужно получить номер текущего месяца от даты начала
    months_count = relative_date.years * 12 + relative_date.months
    
    return months_count

In [4]:
start_date = '2020-01-01'
mon_num = get_mon_num_from_date(start_date)
print(mon_num)

360


In [59]:
# создаем фильтр, который будем применять во воложенном запросе для фильтрации ВСЕХ ТАБЛИЦ по ВСЕМ ИСТОЧНИКАМ
# этот фильтр используем в ТВ, Радио, ООН, Пресса
# Префикс для фильтруемой таблицы задан с запасом = t10
main_filter_str = f't10.sid2 in (2272,2277)'

In [60]:
first_row_query_dict = {
    'tv': 't1.vid, t1.cid, t1.distr, t1.mon, t1.from_mon, t1.estat, t1.cnd_cost_rub,  t1.vol, t1.cnt'
}

In [9]:
media_type = 'tv'
query = f"""select 
distinct t3.sbid
from {media_type}_Ad_month t1 left join {media_type}_Appendix t10 
on t1.vid=t10.vid
left join tv_Appendix t3
on t1.vid=t3.vid
where t1.mon>={str(mon_num)}  and 
({main_filter_str})"""
# отправляем запрос в БД Медиа инвестиции

df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

Все ок. Подключились!
Загрузка завершена успешно


In [10]:
df.shape

(852, 1)

In [12]:
file_path = r'C:\Users\o.bogomolov\Downloads\Telegram'
file_name = 'subbrand_id.xlsx'
df.to_excel(os.path.join(file_path, file_name))

In [ ]:
import os
import pandas as pd

file_path = r'C:\Users\o.bogomolov\Downloads\Telegram'
file_name = 'simple_02_01_2023.xlsx'
df.to_excel(os.path.join(file_path, file_name))

In [61]:
import os
import gc
import sys
import re
import json
from datetime import datetime, date, timedelta, time
import time
from dateutil.relativedelta import relativedelta
import warnings
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import pyodbc
import turbodbc
from turbodbc import connect

from IPython.display import JSON
from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

In [62]:
# добавляем библиотеки для работы с ТВ индексом

%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import time
import pandas as pd
import numpy as np
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 314
Всего найдено записей: 314

Запрошены записи: 0 - 314
Всего найдено записей: 314



In [63]:
# Статистики для отчета Buying
nat_tv_bying_statistics = ['SalesRtgPer', 'StandSalesRtgPer', 'ConsolidatedCostRUB', 'Quantity']

In [64]:
nat_tv_buying_slices = nat_tv_slices = [
    
    #'articleLevel3Id', 'articleLevel4Id',
    'advertiserListId', #Список рекламодателей id
    'advertiserListName', 
    'brandListId', #Список брендов id
    'brandListName', 
    'brandId', #Список брендов id
    'brandName',
    'subbrandListId', #Список суббрендов id
    'subbrandListName', 
    'subbrandId', #Список суббрендов id
    'subbrandName',
    'modelListId', #Список продуктов id
    # 'modelListName',
    'articleList2Id', #Список товарных категорий 2 id
    'articleList2Name',
    'articleLevel2Id',
    'articleLevel2Name',
    # 'articleList3Id', #Список товарных категорий 3 id
    # # 'articleList3Name',
    # 'articleList4Id', #Список товарных категорий 4 id
    # 'articleList4Name', 
    'adId',
    'adName',
    # 'adNotes',
    # 'adFirstIssueDate',
    # 'adSloganAudioId', #Ролик аудио слоган id
    # # 'adSloganAudioName',
    # 'adSloganVideoId', #Ролик видео слоган id
    # 'adSloganVideoName',
    'researchDate',
    # 'regionId', # ИД Региона или Сетевое вещание
    # # 'regionName',
    # 'adDistributionTypeName',
    # # 'tvNetId', # id телесети
    # # 'tvNetName',
    'tvCompanyId',
    # # 'tvCompanyName',
    # 'adTypeId', #Ролик тип ID 
    # # 'adTypeName',
    # 'adStandardDuration', 
    # # 'adPositionType', #Ролик позиционирование id 
    # 'adPositionTypeName',
    # # 'adPrimeTimeStatusId', #Прайм/ОфПрайм роликов id
    # 'adPrimeTimeStatusName',
    # 'adStartTime',
    'adSpotId'
         ]

In [67]:
ad_filter = config.nat_tv_ad_filter + ' and adId in (4721298)' # ad_filter
print(ad_filter) 

articleLevel2Id IN (2277, 2272, 3972, -11, -126) and adIssueStatusId = R and adDistributionType IN (N,O) and adId in (4721298)


In [48]:
from config_search_funcs import get_subbrand_id_str
subbrand_id_str = get_subbrand_id_str(mon_num='360', media_type='tv')
ad_filter = f'{base_filter} AND subbrandId IN ({subbrand_id_str})'
# print(ad_filter)

Все ок. Подключились!
Загрузка завершена успешно
articleLevel2Id IN (2277, 2272, 3972, -11, -126) and adIssueStatusId = R and adDistributionType IN (N,O) and adId in (4571350, 4571551, 4573069, 4573103, 4573233, 4573767, 4189721) AND subbrandId IN (716787, 721281, 581493, 618542, 152306, 889350, 853186, 45840, 850029, 1018055, 26118, 313116, 1025732, 427394, 1233495, 499150, 15447, 1110175, 1068601, 999255, 1248844, 1143271, 1077870, 628001, 843000, 992658, 814204, 35159, 120838, 1006538, 1144677, 168157, 898953, 641773, 766639, 993846, 14129, 1091794, 342189, 1214012, 1154130, 1237762, 782904, 1044140, 842612, 319479, 1008039, 419828, 1083296, 845183, 1095722, 13632, 400724, 1135270, 881376, 1202466, 1066860, 1139240, 924379, 298109, 831750, 15785, 946729, 666970, 617610, 642588, 894570, 926701, 1124041, 1266357, 9998, 298187, 1035105, 1050465, 930337, 800310, 1151049, 1255632, 529614, 18973, 1081578, 44778, 888675, 75455, 758236, 332486, 945907, 400444, 939229, 1201756, 1127982, 8841

In [68]:
import config_tv_index
# список срезов, по которым будет разбивка отчета
slices = nat_tv_buying_slices#config.nat_tv_buying_slices
# список метрик для отчета Simple
statistics = nat_tv_bying_statistics
# Здесь указаны логические условия ad_filter 
# Они применяются для получения статистики в отчетах Simple и Buying# условия фильтрации для запроса 
ad_filter = ad_filter  #config.nat_tv_ad_filter
# Опции для расчета - вся рф и тд.
options = config_tv_index.nat_tv_options

In [69]:
weekday_filter = None
# Задаем тип дня
daytype_filter = None
# Задаем временной интервал
time_filter = None
# Задаем ЦА
basedemo_filter = None
# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None
# Задаем место просмотра
location_filter=None
# Задаем каналы
company_filter = None
# Указываем фильтр программ: продолжительность от 5 минут (300 секунд) 
program_filter = None
# Фильтр блоков
break_filter = None
sortings = None

In [70]:
start_date = '2024-12-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d').date()

end_date = '2024-12-31'
end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

print(f'start_date: {start_date} / end_date: {end_date}')

start_date: 2024-12-01 / end_date: 2024-12-31


In [71]:
 date_filter = [(str(start_date), str(end_date))]

In [72]:
# Посчитаем задания в цикле
tasks = []
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                    daytype_filter=daytype_filter, company_filter=company_filter, 
                                    location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                    targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                    break_filter=break_filter, ad_filter=ad_filter, 
                                    slices=slices, statistics=statistics, sortings=sortings, options=options)


# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_simple_task(task_json))
# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

Расчет задачи (id: 7954d239-1837-402e-a96e-05e3a926cbbd) [= = = = = = = = = = = = = = = = = ] время расчета: 0:00:52.976290


In [73]:
df.shape

(383, 24)

In [75]:
df.head(1)

,modelListId,tvCompanyId,articleList2Id,articleLevel2Id,advertiserListName,subbrandName,subbrandListId,articleLevel2Name,brandListName,adId,adName,subbrandListName,brandName,articleList2Name,subbrandId,brandId,advertiserListId,brandListId,adSpotId,researchDate,SalesRtgPer,StandSalesRtgPer,Quantity,ConsolidatedCostRUB
0,907295,1871,3607,3972,СБЕР,СБЕР ПРАЙМ,730153,УСЛУГИ В ОБЛАСТИ ИНТЕРНЕТА,OKKO (КИНОТЕАТР); САМОКАТ (ДОСТАВКА); СБЕР,4721298,"СБЕР ПРАЙМ подписка на сервисы (Лучше с ПРАЙМом, ВНЧ девушка встречает курьера) ВНЧ. 1 стр длиннее",OKKO.TV; САМОКАТ (ДОСТАВКА); СБЕР ПРАЙМ,СБЕР,СРЕДСТВА МАССОВОЙ ИНФОРМАЦИИ; УСЛУГИ В ОБЛАСТИ ИНТЕРНЕТА; УСЛУГИ В ОБЛАСТИ ТОРГОВЛИ,1081656,1054587,954,674985,6076222304,2024-12-20,0.236532,0.236532,1.0,18730.0


In [77]:
df[['subbrandId']].drop_duplicates()

,subbrandId
0,1081656


In [ ]:
df['ConsolidatedCostRUB'].sum()

In [1]:
import datetime

launch_date = datetime.date(day=10, month=2, year=2022)
print(launch_date)

2022-02-10


In [2]:
some_future = launch_date + datetime.timedelta(days=42)
print(some_future)

2022-03-24
